### Generating lyrics
For trainiing a GPT-2 model we will use gpt-2-simple, which is makes it easy to retrain a model with Wu-Tang lyrics. Go check https://github.com/minimaxir/gpt-2-simple out!

In [ ]:
import gpt_2_simple as gpt2
import os
import requests

model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/
file_name = "wu_tang_songs.txt"

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              steps=20)   # steps is max number of training steps

In [ ]:
gpt2.generate(sess, temperature=1.0)